In [9]:
using SHA

function char_to_bin(c::Char)
    return string(UInt8(c), base=2)
end

function string_to_bin(str)
    return join([char_to_bin(c) for c in str])
end

function bin_to_string(bin_str)
    return join([Char(parse(UInt8, bin_str[i:min(i+7, end)] , base=2)) for i in 1:8:length(bin_str)])
end

function sha1_union(strings::Array{String, 1})
    bin_strings = [string_to_bin(str) for str in strings]

    bin_union = bin_strings[1]
    for i in 2:length(bin_strings)
        bin_union = string(parse(BigInt, "0b" * bin_union) | parse(BigInt, "0b" * bin_strings[i]), base=2)
    end

    str_union = bin_to_string(bin_union)
    new_sha1_hash = bytes2hex(SHA.sha1(str_union))

    return new_sha1_hash
end

function sha1_intersection(strings::Array{String, 1})
    bin_strings = [string_to_bin(str) for str in strings]

    bin_intersection = bin_strings[1]
    for i in 2:length(bin_strings)
        bin_intersection = string(parse(BigInt, "0b" * bin_intersection) & parse(BigInt, "0b" * bin_strings[i]), base=2)
    end

    str_intersection = bin_to_string(bin_intersection)
    new_sha1_hash = bytes2hex(SHA.sha1(str_intersection))

    return new_sha1_hash
end

strings = [ 
        "aba567b86136d0ad7cdaec68e600ca02a3400544", 
        "g1a567b86136d0ad7cdaec68e600ca02a3400530",
        "b1a567b86136d0ad7cdaec68e600ca02a3400526"]

println(sha1_union(strings))
println(sha1_intersection(strings))

c9847d0263de186fc4e3f56b648895565bcd5952
a899d4a28e2c26ae888e964aebec6bfedf30b806
